In [125]:
import os

In [126]:
%pwd

'D:\\ML projects\\ML class\\Kidney-Disease-Classification'

In [127]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path

In [128]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories
from pathlib import Path

In [129]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath= PARAMS_FILE_PATH,
                 ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.path_local = params_filepath
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config

In [130]:
import os
import urllib.request as request
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [131]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self) -> str:
        """
        Fetch data from URL
        :return: 
        """
        
        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")
            
            file_id=dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)
            
            logger.info(f"Download data from {dataset_url} into file {zip_download_dir}")
        
        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        :return: 
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [132]:
try:
    config=ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-05-22 22:46:35,171:INFO:common:yaml file: config\config.yaml loaded successfully]
[2024-05-22 22:46:35,174:INFO:common:yaml file: params.yaml loaded successfully]
[2024-05-22 22:46:35,176:INFO:common:created directory at: artifacts]
[2024-05-22 22:46:35,178:INFO:common:created directory at: artifacts/data_ingestion]
[2024-05-22 22:46:35,180:INFO:906975951:Downloading data from https://drive.google.com/file/d/1SrvZQiGilu4ZbeDO94VL4r2vFpcBRUtY/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1SrvZQiGilu4ZbeDO94VL4r2vFpcBRUtY
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1SrvZQiGilu4ZbeDO94VL4r2vFpcBRUtY&confirm=t&uuid=69a26268-91b2-41f0-895f-d7b4758d4126
To: D:\ML projects\ML class\Kidney-Disease-Classification\artifacts\data_ingestion\data.zip
100%|██████████| 91.8M/91.8M [00:47<00:00, 1.95MB/s]

[2024-05-22 22:47:23,453:INFO:906975951:Download data from https://drive.google.com/file/d/1SrvZQiGilu4ZbeDO94VL4r2vFpcBRUtY/view?usp=sharing into file artifacts/data_ingestion/data.zip]
